# Table of Contents
1. Creating UCCSD ansatz circuit for LiH
2. Optimizing circuit
3. Mapping circuit to nearest neighbor graph
-----
## 1. Creating UCCSD Ansatz Circuit for LiH

In [1]:
import sys
sys.path.append('..')
from fqc import uccsd, util
import numpy as np

In [2]:
theta_vector = [np.random.rand() * 2 * np.pi for _ in range(8)]
print(theta_vector)

[2.102125314103043, 6.0107825335525975, 5.809125377383729, 2.8621996114520103, 2.7217243107379603, 1.2042893261120373, 3.4466133330880857, 4.355308614631206]


In [3]:
circuit = uccsd.get_uccsd_circuit('LiH', theta_vector)
circuit.draw(output='text', line_length=4000)

In [4]:
util.circuitutil.get_max_pulse_time(circuit)

995.1000000000039

In [5]:
unoptimized_unitary = util.get_unitary(circuit)

## 2. Optimizing circuit

In [6]:
circuit = util.circuitutil.optimize_circuit(circuit)

In [7]:
circuit.draw(output='text', line_length=4000)

In [8]:
util.circuitutil.get_max_pulse_time(circuit)

848.9000000000032

In [9]:
optimized_unitary = util.get_unitary(circuit)
print(np.array_equal(optimized_unitary, unoptimized_unitary))

True


## 3. Mapping circuit to nearest neighbor graph

In [10]:
coupling_list = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2)

In [11]:
mapped_circuit = util.circuitutil.optimize_circuit(circuit, coupling_list)

In [12]:
mapped_circuit.draw(output='text', line_length=4000)

In [13]:
util.circuitutil.get_max_pulse_time(mapped_circuit)

932.1000000000035

In [14]:
from quantum_optimal_control.helper_functions.grape_functions import transmon_gate
from quantum_optimal_control.main_grape.grape import Grape
from quantum_optimal_control.core import hamiltonian

In [15]:
import numpy as np
import os, sys, math
from datetime import datetime
sys.path.append('..'); import config

data_path = config.DATA_PATH
file_name = datetime.today().strftime('%h%d')

In [16]:
d = 2  # this is the number of energy levels to consider (i.e. d-level qudits)
max_iterations = 2000
decay =  max_iterations / 2
convergence = {'rate':0.02, 'max_iterations':max_iterations,
               'conv_target':1e-3, 'learning_rate_decay':decay, 'min_grad': 1e-5}
reg_coeffs = {'dwdt': 0.001, 'envelope': 0.01}

In [17]:
N = 4
connected_qubit_pairs = util.circuitutil.get_nearest_neighbor_coupling_list(2, 2, directed=False)
H0 = hamiltonian.get_H0(N, d)
Hops, Hnames = hamiltonian.get_Hops_and_Hnames(N, d, connected_qubit_pairs)
states_concerned_list = hamiltonian.get_full_states_concerned_list(N, d)
maxA = hamiltonian.get_maxA(N, d, connected_qubit_pairs)
U = transmon_gate(optimized_unitary, d)

In [25]:
total_time = 49.8
steps = int(total_time * 20)
uks, U_f = Grape(H0, Hops, Hnames, U, total_time, steps, states_concerned_list, convergence,
                 reg_coeffs=reg_coeffs,
                 use_gpu=False, sparse_H=False, method='ADAM', maxA=maxA,
                 show_plots=False, file_name=file_name, data_path=data_path)

data saved at: /project/ftchong/qoc/pranav/00057_Mar06.h5
Using 5 Taylor terms and 4 Scaling & Squaring terms
Building graph:
Propagators initialized.
Operators weight initialized.
Intermediate propagation variables initialized.
Intermediate propagators initialized.
Vectors initialized.
Training loss initialized.
Optimizer initialized.
Utilities initialized.
Graph built!
Initialized
Error = :9.94e-01; Runtime: 1.5s; Iterations = 0, grads =   3.105e-04, unitary_metric = 1.00034
Error = :5.18e-01; Runtime: 90.7s; Iterations = 100, grads =   1.737e+00, unitary_metric = 1.00009
Error = :1.29e-01; Runtime: 178.7s; Iterations = 200, grads =   1.223e-01, unitary_metric = 1.00007
Error = :1.74e-02; Runtime: 266.6s; Iterations = 300, grads =   7.648e-02, unitary_metric = 0.99999
Error = :5.56e-03; Runtime: 354.5s; Iterations = 400, grads =   2.207e-02, unitary_metric = 0.99995
Error = :2.16e-03; Runtime: 442.1s; Iterations = 500, grads =   8.145e-03, unitary_metric = 0.99999
Error = :9.56e-04; 